In [26]:
import numpy as np
from matplotlib import pyplot as plt
import pennylane as qml
from mitiq import Executor, Observable, PauliString
from mitiq.interface.mitiq_cirq import compute_density_matrix
from mitiq.cdr import generate_training_circuits
from mitiq.cdr._testing import random_x_z_cnot_circuit
from mitiq.pec import execute_with_pec
from mitiq.pec.representations.depolarizing import (
    represent_operation_with_local_depolarizing_noise,
)
from mitiq.pec.representations.learning import (
    depolarizing_noise_loss_function,
    learn_depolarizing_noise_parameter,
)

In [27]:
n_wires = 4

In [28]:
# Describe noise
noise_strength = 0.04

#Ideal device
dev_ideal = qml.device('default.mixed', wires=n_wires)
#noisy device
dev_noisy = qml.transforms.insert(
    dev_ideal,
    qml.DepolarizingChannel,
    noise_strength
)

In [42]:
n_layers = 1
template = qml.SimplifiedTwoDesign
weights_shape = template.shape(n_layers, n_wires)
w1, w2 = [2 * np.pi * np.random.random(s) for s in weights_shape]


def circuit(w1, w2):
    """This circuit runs an identity transformation, so our expectation value should be 1 in a noiseless setting"""
    template(w1, w2, wires=range(n_wires))
    qml.adjoint(template)(w1, w2, wires=range(n_wires))
    return qml.expval(qml.PauliZ(0))


In [30]:
ideal_qnode = qml.QNode(circuit, dev_ideal)
noisy_qnode = qml.QNode(circuit, dev_noisy)

In [31]:
print(qml.draw(ideal_qnode, expansion_strategy="device")(w1, w2))

0: ──RY(6.24)─╭●──RY(0.56)──RY(-0.56)────────────────────────────────────╭●──RY(-6.24)─┤  <Z>
1: ──RY(2.90)─╰Z──RY(1.93)─╭●──────────RY(4.72)──RY(-4.72)─╭●──RY(-1.93)─╰Z──RY(-2.90)─┤     
2: ──RY(0.93)─╭●──RY(0.28)─╰Z──────────RY(5.32)──RY(-5.32)─╰Z──RY(-0.28)─╭●──RY(-0.93)─┤     
3: ──RY(2.81)─╰Z──RY(0.30)──RY(-0.30)────────────────────────────────────╰Z──RY(-2.81)─┤     


In [32]:
ideal_value = ideal_qnode(w1, w2)
print(ideal_value)

0.9999999999999994


In [33]:
noisy_value = noisy_qnode(w1, w2)
print(noisy_value)

0.7192687725433475


In [48]:
circuit = qml.tape.QuantumTape([
    template(w1, w2, wires=range(n_wires)),
    qml.adjoint(template(w1, w2, wires=range(n_wires)))
])

In [35]:
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

23 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.


In [36]:
print(reps[0])

q_2: ───Ry(-0.089π)─── = 1.010*(q_2: ───Ry(-0.089π)───)-0.003*(q_2: ───Ry(-0.089π)───X───)-0.003*(q_2: ───Ry(-0.089π)───Y───)-0.003*(q_2: ───Ry(-0.089π)───Z───)


In [46]:
from mitiq import Executor, QPROGRAM, QuantumResult
import inspect

print(QPROGRAM)
print(type(ideal_qnode))
print(inspect.getfullargspec(ideal_qnode).annotations["return"])
executable = Executor(ideal_qnode)

typing.Union[cirq.circuits.circuit.Circuit, pyquil.quil.Program, qiskit.circuit.quantumcircuit.QuantumCircuit, pennylane.tape.tape.QuantumTape]
<class 'pennylane.qnode.QNode'>
~Result


In [49]:
from mitiq import pec

pec_value = pec.execute_with_pec(circuit, executable, representations=reps)

print(f"Error without PEC: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC:    {abs(ideal_value - pec_value) :.5f}")

/Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q_0: ───Ry(0.92π)───.
  warnings.warn(
/Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q_1: ───Ry(0.733π)───.
  warnings.warn(
/Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q_2: ───Ry(0.374π)───.
  warnings.warn(
/Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q_3: ───Ry(0.332π)───.
  warnings.warn(
/Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages/mitiq/pec/sampling.py:75: UserWarning: No representation found for 

q_0: ─

TypeError: circuit() missing 1 required positional argument: 'w2'